### FastText
- 페이스북에서 개발
- Word2Vect 확장 모듈(임베딩모델)
- 워드 임베딩에 N-gram 적용
- word2vect와 차이점
    - word2vect : 단어(token)는 쪼개질 수 없는 단위로 생각
    - fasttext : 하나의 단어 안에 여러 단어들이 존재하는 것으로 간주
        - 내부 단어, 서부워드를 고려해서 학습
- fasttext 특징
    - 내부 단어 학습(철자별로 해버림)
    - 모르는 단어에 대한 대응
    - 빈도수가 적었던 단어에 대한 대응

### 내부 단어 학습
- N-gram 방식을 도입 : 단어에 문자 단위로 적용
- 'hello' : bi-gram 적용 (he, el, ll, lo) 분리하고 벡터로 생성
- 'apple' : n=3 (ap, app, ppl, ple, le)

### 모르는 단어 : oov(out of vocabulary)에 대한 대응
- birthplace : oov 라고 한다면
    - birth // place 나눠서 사전에서 탐색 >> 두 단어 임베딩벡터 결합해서 birthplace 얻을 수 있음

### 빈도수가 적은 단어
- 오타인 경우 빈도수가 적다고 가정
- 오타인 단어를 쪼개서 오타가 아닌 부분을 gram으로 사용해서 사전에서 확인 >> 해당 임베딩을 사용했을 때, 성능 향상
- birth >> 오타 : bith >> bi, th

In [1]:
# 데이터 전처리 
from nltk.tokenize import  sent_tokenize, word_tokenize
from lxml import etree
import re

targetXML = open('./data/ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [2]:
len(result)

273424

### word2vec와 fasttext 차이

In [5]:
# result 데이터로 학습한 word2vec 모델 생성
from gensim.models import Word2Vec, FastText
model_wv = Word2Vec(sentences=result, vector_size =100, window=5, workers=4, sg=0)

In [7]:
model_wv.wv["king"]
model_wv.wv.most_similar("king")

[('martin', 0.8728253841400146),
 ('luther', 0.8509300351142883),
 ('james', 0.8490146398544312),
 ('thomas', 0.8405089378356934),
 ('john', 0.8357966542243958),
 ('president', 0.828530490398407),
 ('dr', 0.8105995059013367),
 ('charles', 0.8077185750007629),
 ('george', 0.8053943514823914),
 ('alexander', 0.8024201393127441)]

In [9]:
# 단어 사전에 없는 단어(oov) : word2vec에선 에러 발생 >> KeyError: "Key 'electrofishing' not present in vocabulary"
model_wv.wv.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

### FastText
- 단어들에 n-gram 적용

In [10]:
model_ft = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [12]:
model_ft.wv.most_similar("king")

[('viking', 0.9769628047943115),
 ('biking', 0.9735829830169678),
 ('joking', 0.9715903997421265),
 ('hiking', 0.9700593948364258),
 ('spiking', 0.9686288833618164),
 ('pecking', 0.9664033651351929),
 ('poking', 0.9660462141036987),
 ('cranking', 0.9644487500190735),
 ('lacking', 0.9639366269111633),
 ('hawking', 0.9636201858520508)]

In [13]:
model_ft.wv.most_similar("electrofishing")

[('fishing', 0.9170586466789246),
 ('licensing', 0.9080235958099365),
 ('flashing', 0.9062708616256714),
 ('flushing', 0.9054815769195557),
 ('flourishing', 0.9031326770782471),
 ('refreshing', 0.8977901339530945),
 ('smashing', 0.8968352675437927),
 ('vanishing', 0.8957969546318054),
 ('operating', 0.8953908085823059),
 ('transplanting', 0.8943449854850769)]

### 한국어 fasttext
- 한글의 경우에도 oov문제를 해결하기 위해 fasttext 적용 시도
- 음절 단위로 n-gram 적용 ( word2vec)
    - n=3, "자연어처리"에 대한 n-gram
    - <자연, 자연어, 연어처, 어처리, 처리> 와 같은 방식으로 n-gram이 적용
- 자모단위 ( fasttext)
    - 초성,중성,종성 단위로 임베딩
    - 오타가 많은 데이터에서 더 강한 임베딩 성능 기대 가능
    - "자연어처리"에 대한 n-gram : 자모단위
    - n=3, <ㅈ ㅏ, ㅈ ㅏ _(받침이 없는 경우 받침 없는 걸 표현), ㅏ _ ㅇ, ㅇ ㅕ ㄴ,
            ㅕ ㄴ ㅇ, ...> 의 형태로
- 한글 자모단위 처리 패키지
    - pip install hgtk

In [14]:
import hgtk

In [18]:
# 한글 확인 
hgtk.checker.is_hangul("ㄱ")
# hgtk.checker.is_hangul("2")

True

In [22]:
# hgtk.letter.decompose("letter") # 한글 아니라 에러
hgtk.letter.decompose("남")

('ㄴ', 'ㅏ', 'ㅁ')

In [23]:
cho, jung, jong = hgtk.letter.decompose("여")
cho, jung, jong 

('ㅇ', 'ㅕ', '')

In [25]:
# 자모단위 결합 함수
hgtk.letter.compose("ㄴ", "ㅏ")

'나'

In [27]:
# 결합할 수 없는 상황에서는 에러 : NotHangulException: No valid Hangul character index
# hgtk.letter.compose("ㄴ", "ㅁ", "ㅇ") 

### 네이버 쇼핑 리뷰 데이터 사용

In [28]:
import pandas as pd
total_data = pd.read_table("./data/ratings_total.txt", names= ["ratings", "review"])
total_data.head()

,ratings,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [31]:
# # 특정 단어를 초성,중성,종성으로 나누는함수

# def word_to_jamo(token):
#   def to_special_token(jamo):
#     if not jamo:
#       return '-'
#     else:
#       return jamo

#   decomposed_token = ''
#   for char in token:
#     try:
#       # char(음절)을 초성, 중성, 종성으로 분리
#       cho, jung, jong = hgtk.letter.decompose(char)

#       # 자모가 빈 문자일 경우 특수문자 -로 대체
#       cho = to_special_token(cho)
#       jung = to_special_token(jung)
#       jong = to_special_token(jong)
#       decomposed_token = decomposed_token + cho + jung + jong

#     # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
#     except Exception as exception:
#       if type(exception).__name__ == 'NotHangulException':
#         decomposed_token += char
    
#   # 단어 토큰의 자모 단위 분리 결과를 추가
#   return decomposed_token

In [32]:
# 특정 단어를 초, 중, 종성으로 나누는 사용자 정의 함수
def word_to_jamo(token) :
    def to_special_token(jamo) :
        if not jamo :
            return "-"
        else :
            return jamo
    decomposed_token = ""
    for char in token :
        try :
            cho, jung, jong = hgtk.letter.decompose(char) # 받침이 없으면 jong은 ""가 입력됨
            # 자모가 빈 문자열일 경우, 특수문자 "-"로 대체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong
        except Exception as exception :
            if type(exception).__name__ == "NotHangulException":
                decomposed_token += char
    return decomposed_token

In [33]:
word_to_jamo("남동생")

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [34]:
word_to_jamo("여동생")

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [36]:
from konlpy.tag import Okt
okt = Okt()
okt.morphs("선물용으로 빨리 받아서 전달 했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.")

['선물',
 '용',
 '으로',
 '빨리',
 '받아서',
 '전달',
 '했어야',
 '하는',
 '상품',
 '이었는데',
 '머그컵',
 '만',
 '와서',
 '당황',
 '했습니다',
 '.']

In [39]:
def tokenize_by_jamo(sentence) :
    return [word_to_jamo(token) for token in okt.morphs(sentence)]

In [42]:
sentence = "선물용으로 빨리 받아서 전달 했어야 하는 상품이었는데 머그컵만 와서 당황했습니다."
print(tokenize_by_jamo(sentence))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-ㅇㅓㅆㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [47]:
%%time
from tqdm import tqdm

tokenized_data = [] # 리부 전체 문장 토큰 저장할 리스트
for sample in tqdm(total_data["review"].to_list()):
    tokenized_sample = tokenize_by_jamo(sample)
    tokenized_data.append(tokenized_sample)

100%|████████████████████████████| 200000/200000 [1:05:18<00:00, 51.04it/s]

CPU times: total: 1h 5min 40s
Wall time: 1h 5min 18s


In [48]:
len(tokenized_data)

200000

In [49]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-ㄱㅗ-', 'ㄱㅜㅅ']

In [51]:
# 단어로 복원해주는 함수 

def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0
  
  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']
  
  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo
    # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:  
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

### 한글 FastText 임베딩 모델 생성
- pip install fasttext-wheel

In [52]:
import fasttext

In [53]:
# tokenized_data(list)를 말뭉치(텍스트 집합)으로 다시 구성해야함 >> 즉, list 요소를 txt파일로 저장
with open("./data/tokenized_data.txt", "w", encoding="UTF8") as out :
    for line in tqdm(tokenized_data, unit=" line"):
        out.write(" ".join(line) + "\n")


100%|███████████████████████| 200000/200000 [00:00<00:00, 356555.42 line/s]


In [54]:
# fasttext.train_unsupervised >> 말뭉치를 주입받아서 fasttext 방식으로 임베딩벡터 생성하는 모듈
model_ft_kor = fasttext.train_unsupervised("./data/tokenized_data.txt", model="cbow")

In [55]:
# model_ft_kor은 초/중/종성이 펼쳐진 데이터로 학습을 진행 >> 임베딩벡터값 확인하기 위해서는 초/중/종성이 펼쳐진 데이터로 확인 
model_ft_kor[word_to_jamo("남동생")]

array([ 0.352271  , -0.01096702,  0.06772958, -0.49084365, -0.91934633,
        0.67120206, -0.06407845,  0.90819466, -0.49045715, -0.02728405,
       -0.51844776, -0.9557729 , -0.0447108 , -0.424039  ,  0.45476815,
        0.5911588 ,  0.76276433, -0.3132864 , -0.04146839, -0.7162987 ,
       -0.07241749, -1.371207  , -0.31391716, -0.73100954, -0.21320039,
        1.0922103 ,  0.31125253,  0.4317013 ,  0.9374813 ,  0.19062869,
       -0.36007923,  0.5329514 , -0.5572704 ,  0.33488792, -0.6640924 ,
        0.47277847, -0.23200971,  1.7224989 ,  0.5872937 ,  0.08474042,
       -1.3061486 , -0.07197918, -0.07680016,  0.8138374 ,  1.3192397 ,
        0.12671499, -0.22916922,  0.9782795 ,  0.6477222 , -0.28950304,
       -0.3767516 ,  0.28815356,  0.43184236, -0.19069389,  0.3739281 ,
       -0.1409797 ,  0.33875376, -0.14300728,  0.6732585 , -0.6241671 ,
        0.6535004 ,  0.2698578 ,  0.8960673 ,  0.47058603, -0.85397184,
       -0.34458378,  0.43461263, -0.23304343, -0.63023025, -0.63

In [56]:
word_to_jamo("남동생")

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [57]:
model_ft_kor["ㄴㅏㅁㄷㅗㅇㅅㅐㅇ"]

array([ 0.352271  , -0.01096702,  0.06772958, -0.49084365, -0.91934633,
        0.67120206, -0.06407845,  0.90819466, -0.49045715, -0.02728405,
       -0.51844776, -0.9557729 , -0.0447108 , -0.424039  ,  0.45476815,
        0.5911588 ,  0.76276433, -0.3132864 , -0.04146839, -0.7162987 ,
       -0.07241749, -1.371207  , -0.31391716, -0.73100954, -0.21320039,
        1.0922103 ,  0.31125253,  0.4317013 ,  0.9374813 ,  0.19062869,
       -0.36007923,  0.5329514 , -0.5572704 ,  0.33488792, -0.6640924 ,
        0.47277847, -0.23200971,  1.7224989 ,  0.5872937 ,  0.08474042,
       -1.3061486 , -0.07197918, -0.07680016,  0.8138374 ,  1.3192397 ,
        0.12671499, -0.22916922,  0.9782795 ,  0.6477222 , -0.28950304,
       -0.3767516 ,  0.28815356,  0.43184236, -0.19069389,  0.3739281 ,
       -0.1409797 ,  0.33875376, -0.14300728,  0.6732585 , -0.6241671 ,
        0.6535004 ,  0.2698578 ,  0.8960673 ,  0.47058603, -0.85397184,
       -0.34458378,  0.43461263, -0.23304343, -0.63023025, -0.63

In [58]:
# 남동생 벡터와 가장 유사도가 높은 벡터 확인
model_ft_kor.get_nearest_neighbors(word_to_jamo("남동생"), k=10)

[(0.83176589012146, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.8244485855102539, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8109169006347656, 'ㄴㅏㅁㅈㅏ-ㅊㅣㄴㄱㅜ-'),
 (0.7926679849624634, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7442033290863037, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.7185494899749756, 'ㅊㅣㄴㄱㅜ-'),
 (0.7138498425483704, 'ㅊㅗ-ㄷㅡㅇㅅㅐㅇ'),
 (0.6991846561431885, 'ㄸㅏㄹㄹㅐㅁ'),
 (0.6507283449172974, 'ㅍㅕㅇㅅㅐㅇ'),
 (0.6504088044166565, 'ㅇㅕ-ㅈㅏ-ㅊㅣㄴㄱㅜ-')]

In [59]:
# jamo_to_word() 함수 사용해서 자모 분해된 것을 단어로 복워해서 출력 
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [60]:
transform(model_ft_kor.get_nearest_neighbors(word_to_jamo("남동생"), k=10))

[('남친', 0.83176589012146),
 ('동생', 0.8244485855102539),
 ('남자친구', 0.8109169006347656),
 ('남편', 0.7926679849624634),
 ('난생', 0.7442033290863037),
 ('친구', 0.7185494899749756),
 ('초등생', 0.7138498425483704),
 ('딸램', 0.6991846561431885),
 ('평생', 0.6507283449172974),
 ('여자친구', 0.6504088044166565)]

In [61]:
transform(model_ft_kor.get_nearest_neighbors(word_to_jamo("고품질"), k=10))

[('음질', 0.7979444265365601),
 ('품질', 0.7903384566307068),
 ('땜질', 0.7696232795715332),
 ('찜질', 0.7372652292251587),
 ('질', 0.7172016501426697),
 ('화질', 0.6837594509124756),
 ('좀질', 0.6759639978408813),
 ('질질', 0.6579735279083252),
 ('질환', 0.6554113626480103),
 ('빗질', 0.6542677879333496)]

In [62]:
transform(model_ft_kor.get_nearest_neighbors(word_to_jamo("보품질"), k=10))

[('보풀', 0.7431467175483704),
 ('보질', 0.7429668307304382),
 ('음질', 0.7326189279556274),
 ('보폴', 0.7292374968528748),
 ('후질', 0.7229625582695007),
 ('화질', 0.7215917706489563),
 ('보이질', 0.7149317264556885),
 ('품질', 0.7018211483955383),
 ('재질', 0.6830146908760071),
 ('찜질', 0.6786102652549744)]

In [63]:
transform(model_ft_kor.get_nearest_neighbors(word_to_jamo("제품"), k=10))

[('반제품', 0.8767932057380676),
 ('재품', 0.8449320793151855),
 ('상품', 0.8114844560623169),
 ('명품', 0.7820002436637878),
 ('신제품', 0.7724065184593201),
 ('중품', 0.7699512243270874),
 ('리퍼제품', 0.7655267119407654),
 ('검품', 0.7478256225585938),
 ('파품', 0.7447069883346558),
 ('재고품', 0.7431009411811829)]

#### FastText 임베딩 벡터는 오타와 노이즈 단어에 꽤 높은 대응을 진행
- 딥러닝 모델 성능 향상 역할